<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/College_essays_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 特定人格作文自动生成与评估

这个脚本由闫晨曦开发，用于研究能否基于GPT生成具有特定人格特点的高考作文并通过人格评估

脚本由李鲁鲁修改

项目链接[Chat凉宫春日](https://github.com/LC1332/Chat-Haruhi-Suzumiya)



食用指南：

- 输入标题，或者选择示例标题

- 选择高/低开放性人格

- Roll keywords 随机选择相应开放性人格的关键词

- 选择6个特征中的三个

- 生成prompt

- 生成作文

- 如果生成的作文，“机”味过浓，比如 “作为一个高开放性人格的人...” 可选择对作文进行Refine

- 评估作文，对生成作文进行人格特性评估

In [1]:
!pip -q install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.3 MB/s eta 0:00:00


In [2]:
import os
import openai

openai.api_key = 'sk-lfrdoJK' # 在这里输入你的OpenAI API Token

os.environ["OPENAI_API_KEY"] = openai.api_key


In [3]:
import json

PROMPT = """你扮演一名资深的心理学家，对于心理学中的人格分析非常了解。

我正在为心理学的课程准备案例，我希望构造一个典型的“低开放性人格”的被试，并让他写一篇500个单词的Essay。

我希望他有如下人格特点：
```
"""

In [4]:
REFINE_PROMPT = """
- 对引号中的作文，用更高质量的词汇改写
- 如果作文中出现"作为一个xx的人"，或者"我是一个xx人格的人"，用自己的切身体会，如“我常常xxx","我经常xxx"的句子进行改写
-如果作文中学生直接表达自己是怎样的人格，为学生替换为更具体的例子。
-请不要在作文中讨论高考或者讨论作文题，如果出现类似的句子，请进行改写或者去除。
-如果作文中出现"高开放性人格"或者"低开放性人格"，去掉这样的句子，或者改为更详细的例子来说明。

请按照上述的规则改写下面的文本:
```
"""

In [5]:
EVALUATE_PROMPT = """
你扮演一个资深的心理学家

我正在设计一个心理学的实验，我希望通过一篇作文，去评估被试在大五人格中 Openness的程度，注意到openness可以被分为Fantasy, Aesthetics, Feelings, Actions, Ideas和Values六个维度。六个维度的解释如下

Fantasy. People who are high in Fantasy are imaginative. They daydream constantly. They ask themself “what if?” They like fiction, and then once they finish a story they’ll start wondering what would have happened if one of the characters had made a different decision.

Aesthetics. People who are high in Aesthetics care deeply about beauty. They love poetry, music, painting, sculpture, dance, and art of all kinds.

Feelings(Openness to Feelings). People who are high in Feelings have more intense emotions. They think emotions are an important part of life. They have a rich emotional vocabulary and nuanced emotional experience.

Actions(Openness to Actions). People who are high in Openness to Actions like trying new things. They like travel, new hobbies, and new foods. If you are outraged that there are places you aren’t going to go and skills you aren’t going to learn and activites you aren’t going to try

Ideas(Openness to Ideas). People who are high in Openness to Ideas are curious. They enjoy thinking, instead of finding it burdensome and laborous. They like philosophical arguments, puzzles, and nonfiction aimed at the educated layperson.

Values(Openness to Values). People who are high in Openness to Values are liberals in the political philosophy sense, not the vote-for-Democrats sense, although they often vote for Democrats. They are tolerant of people different than them. They question tradition. They believe in freedom.

我邀请了一个 被试 ，并让他写了一篇作文：
```
"""
EVALUATE_PROMPT_SUFFIX = "请为我判断 被试 是属于高开放性人格还是低开放性人格\n"

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

李鲁鲁：这里修改为从ChatHaruhi项目获取数据

In [8]:
!wget -q https://raw.githubusercontent.com/LC1332/Chat-Haruhi-Suzumiya/main/characters/personality-data/essays_keywords_high.jsonl
!wget -q https://raw.githubusercontent.com/LC1332/Chat-Haruhi-Suzumiya/main/characters/personality-data/essays_keywords_low.jsonl

In [9]:
low_keywords_file = '/content/essays_keywords_low.jsonl'
high_keywords_file = '/content/essays_keywords_high.jsonl'

In [11]:
#@title 读取keywords
import json
low_keywords = []
high_keywords = []
with open(low_keywords_file, 'r') as f:
    for line in f:
        data = json.loads(line)
        for item in data['keyword']:
            low_keywords.append(item)
    low_keywords = set(low_keywords)
with open(high_keywords_file, 'r') as f:
    for line in f:
        data = json.loads(line)
        for item in data['keyword']:
            high_keywords.append(item)
high_keywords = set(high_keywords)

李鲁鲁：改为从ChatHaruhi获取文件

In [12]:
!wget -q https://raw.githubusercontent.com/LC1332/Chat-Haruhi-Suzumiya/main/characters/personality-data/12_third_personality_prompt.txt

In [13]:
#@title 读取prompts
# 读取prompt

personality_prompts_file = '/content/12_third_personality_prompt.txt'

personality_prompts = {}
texts = []
with open(personality_prompts_file, 'r') as f:
    for line in f.readlines():
        line = line.strip()
        if '-' in line:
            key = line
        elif '。' in line:
            texts.append(line)
        else:
            personality_prompts[key] = texts
            texts = []

In [14]:
# 关键词从high/low里随机选
# 先使用 simale prompt测试，跑通流程

In [15]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.0 MB/s eta 0:

In [20]:
#@title 高考作文例子

gaokao_eamples = ["“本手、妙手、俗手”是围棋的三个术语。本手是指合乎棋理的正规下法；妙手是指出人意料的精妙下法；俗手是指貌似合理，而从全局看通常会受损的下法。"
                 "对于初学者而言，应该从本手开始，本手的功夫扎实了，棋力才会提高。一些初学者热衷于追求妙手，而忽视更为常用的本手。本手是基础，妙手是创造。一般来说，"
                 "对本手理解深刻，才可能出现妙手；否则，难免下出俗手，水平也不易提升。以上材料对我们颇具启示意义。请结合材料写一篇文章，体现你的感悟与思考。",
                 "烟火气是家人团坐，灯火可亲；烟火气是国泰民丰，岁月安好；烟火气是温情，是祥和，需要珍惜和守护，也需要奉献和担当。寻常烟火，就是最美的风景。你对这段话有怎样的思考和感悟？请结合自身体验，写一篇文章",
                  """人们因技术发展得以更好地掌控时间，但也有人因此成了时间的仆人。这句话引发了你怎样的联想与思考？请写一篇文章。""",
                  '''"吹灭别人的灯，并不会让自己更加光明；阻挡别人的路，也不会让自己行得更远。“一花独放不是春，百花齐放春满园。”如果世界上只有一种花朵，就算这种花朵再美，那也是单调的。以上两则材料出自习近平总书记的讲话，以生动形象的语言说出了普遍的道理。请据此写一篇文章，体现你的认识与思考。''',
                  '''"好的故事，可以帮我们更好地表达和沟通，可以触动心灵、启迪智慧；好的故事，可以改变一个人的命运，可以展现一个民族的形象……故事是有力量的。以上材料引发了你怎样的联想和思考？请写一篇文章。''',
                  '''"《红楼梦》写到“大观园试才题对额”时有一个情节，为元妃（贾元春）省亲修建的大观园竣工后，众人给园中桥上亭子的匾额题名。有人主张从欧阳修《醉翁亭记》“有亭翼然”一句中，取“翼然”二字；贾政认为“此亭压水而成”，题名“还须偏于水”，主张从“泻出于两峰之间”中拈出一个“泻”字，有人即附和题为“泻玉”；贾宝玉则觉得用“沁芳”更为新雅，贾政点头默许。“沁芳”二字，点出了花木映水的佳境，不落俗套；也契合元妃省亲之事，蕴藉含蓄，思虑周全。以上材料中，众人给匾额题名，或直接移用，或借鉴化用，或根据情境独创，产生了不同的艺术效果。这个现象也能在更广泛的领域给人以启示，引发深入思考。请你结合自己的学习和生活经验，写一篇文章。''',
                  '''"与有肝胆人共事、从无字句处读书、一代人有一代人的使命与挑战，一代人有一代人的责任和担当。一个世纪前，在津求学的青年周恩来撰写了这副对联，在交友处事与读书求知方面警勉自己。品读此联，你有怎样的联想和思考？请任选角度，结合自己的体验与感悟，写一篇文章。''',
                  '''"如果说时间是一条单行道，那么纪念日就是道路两侧最醒目的路标，它告诉我们怎样从昨天走到了今天。时间永不停步，纪念日不会消失。记住它，可以让日历上简单的数字成为岁月厚重的注脚，而它也不断提醒着我们带着初心奔向前方。你对这段话有怎样的理解和感悟？请结合自身体验，写一篇文章。''',
                  '''"古人常以比喻说明对理想的追求，涉及基础、方法、路径、目标及其关系等。如汉代扬雄就曾以射箭为喻，他说：“修身以为弓，矫思以为矢，立义以为的，奠而后发，发必中矣。”大意是，只要不断加强修养，端正思想，并将“义”作为确定的目标，再付诸行动，就能实现理想。上述材料能给追求理想的当代青年以启示，请结合你对自身发展的思考写一篇文章。''',
                  '''"有人说，经过时间的沉淀，事物的价值才能被人们认识；也有人认为不尽如此。你怎么看？请写一篇文章，谈谈你的思考。''']




In [ ]:
import gradio as gr
import random


def roll_keywords(openness):
    if openness == '高':
        keywords = random.sample(high_keywords, 20)
    else:
        keywords = random.sample(low_keywords, 20)
    return keywords


def assemble_prompt(title, openness, keywords, factors):
    if title == "":
        return {error_box: gr.update(value="Enter title", visible=True)}
    elif keywords == "":
        return {error_box: gr.update(value="Roll keywords", visible=True)}
    elif len(factors) != 3:
        return {error_box: gr.update(value="Please select three features", visible=True)}

    prompts = PROMPT
    # 组装
    sel = 'high' if openness == '高' else 'low'
    for factor in factors:
        for pro in personality_prompts[f"{factor}-{sel}"]:
            prompts += pro + "\n"
        prompts += '\n'
    prompts += '我希望用他的第一人称，围绕用下列关键词：\n'
    prompts += str(keywords) + '\n'
    prompts += '\n今年的高考作文题为\n```\n'
    prompts += title + '\n```\n' + f"构造一篇“{openness}开放性人格”被试的应答这个主题的500个单词的高考作文。可以使用到之前提到的关键词，但不要在作文中特意说明指出自己的开放性人格！"
    return {prompt: prompts}


# GPT模型生成函数
def generate_essay(prompt):
    messages = [
        SystemMessage(content=prompt)
    ]
    # print(prompt)
    return_msg = chat(messages)
    essays = return_msg.content
    return essays


def refine_essays(essays):
    prompt = REFINE_PROMPT
    prompt += '\n' + str(essays) + '\n```'
    messages = [
        HumanMessage(content=prompt)
    ]
    return_msg = chat(messages)
    res = return_msg.content
    return res


def evaluate_essays(essays):
    # evaluate
    evaluate_prompt = EVALUATE_PROMPT
    evaluate_prompt += '\n' + essays + '\n```\n'
    evaluate_prompt += EVALUATE_PROMPT_SUFFIX
    messages = [
        HumanMessage(content=evaluate_prompt)
    ]
    return_msg = chat(messages)
    res = return_msg.content
    return res


def store(title, factors, keywords, essays, evaluate_res=None):
    item = {"title": title, "factors": factors, "keywords": keywords, "essays": essays, "evaluate_res": evaluate_res}
    with open('res.jsonl', 'a') as f:
        json.dump(item, f)
        f.write('\n')


with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 特定人格作文自动生成与评估

        这个脚本由闫晨曦开发

        项目链接[Chat凉宫春日](https://github.com/LC1332/Chat-Haruhi-Suzumiya)

        1. 输入作文题标题，或者选择示例标题 2. 选择高/低开放性人格

        3. 点击 Roll keywords 随机选择相应开放性人格的关键词 4. 选择6个特征中的三个

        5. 点击生成人格prompt 6. 点击生成作文

        7. 如果生成的作文，“机”味过浓，比如 “作为一个高开放性人格的人...” 可选择对作文进行Refine

        8. 评估作文，对生成作文进行人格特性评估
        """)
    error_box = gr.Textbox(label="Log", visible=False)
    with gr.Row():
        with gr.Column():
            title = gr.Textbox(label="作文标题")
            openness = gr.Radio(["高", "低"], label="开放性", value='高'),
            keywords = gr.Textbox(label="关键词")
            factors = gr.CheckboxGroup(["actions", "aesthetics", "fantasy", "feelings", "ideas", "values"],
                                       label="特征",
                                       value=["actions", "aesthetics", "fantasy"])
        prompt = gr.Textbox(label="prompt", lines=8, interactive=True)
    essays = gr.Textbox(label="生成的作文")
    evaluate_res = gr.Textbox(label="评估结果")
    with gr.Row():
        roll_button = gr.Button('Roll keywords')
        prompt_button = gr.Button('生成prompt')
        essay_button = gr.Button("生成作文")
        essay_refine = gr.Button("Refine 作文")
        evaluate_button = gr.Button('评估作文')
    gr.Examples(gaokao_eamples,
                title, label="作文题目示例")
    roll_button.click(fn=roll_keywords, inputs=openness[0], outputs=keywords)
    prompt_button.click(fn=assemble_prompt, inputs=[title, openness[0], keywords, factors], outputs=[error_box, prompt])
    essay_button.click(fn=generate_essay, inputs=prompt, outputs=essays)
    essay_refine.click(fn=refine_essays, inputs=essays, outputs=essays)
    evaluate_button.click(fn=evaluate_essays, inputs=essays, outputs=evaluate_res)

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b377fffe1316451bef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<ipython-input-21-03670cb95077>:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  keywords = random.sample(low_keywords, 20)
